# Testing

## Imports and data

In [31]:
import pandas as pd
import wandb
import xgboost as xgb
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

In [21]:
os.chdir("/Users/antonioslagarias/data-science-code/notebooks")
os.getcwd()

'/Users/antonioslagarias/data-science-code/notebooks'

In [32]:
path_figures = "../figures/milestone2/step6_testing/"
os.makedirs(path_figures, exist_ok=True)

In [23]:
path="/Users/antonioslagarias/data-science-code/data/milestone2/advanced_test.csv"
test_df=pd.read_csv(path)
test_df.head()

,game_id,season,game_type,event_type,period,period_time,x_coord,y_coord,shot_type,team_id,...,team_type,distance_from_net,shot_angle,period_time_seconds,last_event_type,last_event_x,last_event_y,last_event_time_seconds,last_event_distance,time_since_last_event
0,2020020270,2020,general,SHOT_ON_GOAL,1,01:09,65.0,1.0,slap,10,...,home,24.020824,2.385944,69,hit,-93.0,-24.0,NaN,159.965621,6.0
1,2020020270,2020,general,SHOT_ON_GOAL,1,02:21,39.0,31.0,wrist,9,...,away,58.830264,31.798913,141,hit,54.0,39.0,NaN,17.000000,24.0
2,2020020270,2020,general,SHOT_ON_GOAL,1,02:34,42.0,35.0,wrist,9,...,away,58.600341,36.674350,154,shot-on-goal,39.0,31.0,NaN,5.000000,13.0
3,2020020270,2020,general,SHOT_ON_GOAL,1,03:16,84.0,4.0,backhand,9,...,away,6.403124,38.659808,196,hit,84.0,36.0,NaN,32.000000,1.0
4,2020020270,2020,general,SHOT_ON_GOAL,1,04:29,71.0,16.0,backhand,9,...,away,24.083189,41.633539,269,faceoff,-69.0,-22.0,NaN,145.065502,26.0


In [30]:
for i in final_features:
    if i not in list(test_df.columns):
        print(i)

rebound


## Best XGBoost

In [19]:
run = wandb.init(project="milestone_2") 
       
# use your project name
artifact = run.use_artifact('IFT6758_team4/milestone_2/xgboost_model3:v0', type='model')
artifact_dir = artifact.download()
wandb.finish() 

wandb:   1 of 1 files downloaded.  


In [16]:
model_path = f"{artifact_dir}/model3.ubj"  
model = xgb.Booster()
model.load_model(model_path)

In [ ]:
final_features=['distance_from_net', 'rebound', 'period', 
                'last_event_distance', 'shot_angle', 'shot_type', 
                'period_time_seconds']

test_df[final_features]

In [25]:
X_test=test_df[final_features]
y_test=test_df[["is_goal"]]

KeyError: "['rebound'] not in index"

In [ ]:
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
y_pred = model3.predict(dtest)
df_test = pd.DataFrame({
    "y": y_test,
    "p": y_pred
})

In [ ]:
#Compute 
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
# Plot
plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.3f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve: Best Models')
plt.legend(loc='lower right')
plt.grid(True)
#plt.savefig(path_figures+"roc_test.png", dpi=300)
plt.show()

In [ ]:
#compute
df_test["percentile_bin"] = pd.qcut(df_test["p"], q=10, labels=False)

grouped = df_test.groupby("percentile_bin").agg(
    goal_rate=("y", "mean"),
    mean_prob=("p", "mean"),
    count=("y", "size")
).reset_index()

grouped["percentile_label"] = (grouped["percentile_bin"] + 1) * 10
print(grouped.loc[grouped["percentile_bin"].idxmax()])

# Plot (change the labels as you need)
plt.figure(figsize=(7, 5))

plt.plot(
    grouped["percentile_label"],
    grouped["goal_rate"],
    marker="o",
    lw=2,
    label="XGBoost-Model"
)

plt.xlabel("Pred. probability percentile")
plt.ylabel("Goal rate")
plt.title("Percentile: Best Models")
plt.xticks(grouped["percentile_label"], [f"{x}%" for x in grouped["percentile_label"]])
plt.legend(loc="upper right")
plt.grid(True)
plt.gca().invert_xaxis()
#save
#plt.savefig(path_figures + "percentile_test.png", dpi=300)
plt.show()

In [ ]:
# Compute
df_test = df_test.sort_values("p", ascending=False).reset_index(drop=True)
df_test["cum_shots"] = np.arange(1, len(df_test) + 1)
df_test["cum_goals"] = df_test["y"].cumsum()

df_test["cum_shots_pct"] = df_test["cum_shots"] / len(df_test)
df_test["cum_goals_pct"] = df_test["cum_goals"] / df_test["y"].sum()

# Plot
plt.figure(figsize=(7, 5))

plt.plot(
    df_test["cum_shots_pct"] * 100,
    df_test["cum_goals_pct"] * 100,
    color="darkorange",
    lw=2,
    label="XGBoost-Model"
)

plt.xlabel("% of shots (cumulative, sorted by pred.)")
plt.ylabel("% of goals (cumulative)")
plt.title("Cumulative: Best Models")
plt.legend(loc="lower right")
plt.grid(True)

#Save
#plt.savefig(path_figures + "cumulative_test.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Compute
prob_true, prob_pred = calibration_curve(df_test["y"], df_test["p"], n_bins=10)

# Plot
plt.figure(figsize=(7, 5))

plt.plot(
    prob_pred,
    prob_true,
    marker="o",
    color="darkorange",
    lw=2,
    label="XGBoost-Model"
)


plt.plot([0, 1], [0, 1], color="navy", lw=1, linestyle="--", label="")

plt.xlabel("Predicted probability")
plt.ylabel("Observed goal rate")
plt.title("Reliability Curve: Best Models")
plt.legend(loc="lower right")
plt.grid(True)

# Save
#plt.savefig(path_figures + "calibration_test.png", dpi=300, bbox_inches="tight")
plt.show()
